# MiniFlow implementation for Self-driving Car Nanodegree (Udacity)

First of all we will proceed to implement a graph structure in MiniFlow. For this purpose we will create a Python class in order to represent a generic node. 

## Node class

The node class will have the follwing properties:
* each node might receive input from multiple other nodes *self.inbound_nodes)*
* each node creates a single output which will likely be passed to other nodes *(self.outbound_nodes)*
* each node will eventually calculate a value that represents its output *(value)*

For the implementation we will use two lists: one to store references to the inbound nodes, and the other to store references to the outbound nodes. An inbound node is a node from which the current node receives values. Meanwhile, outbound nodes are the nodes to which the current node passes values.

In [10]:
class Node(object):
    def __init__(self, inbound_nodes=[]):
        #Node properties
        self.inbound_nodes = inbound_nodes
        self.outbound_nodes = []
        self.value = None
        
        #For each inbound node, we will add the current node (self) as an outbound node of a given inbound node.
        for innode in self.inbound_nodes:
            innode.outbound_nodes.append(self)

Another property is that each node will need to be able to pass values forward and also perform backpropagation. We will first implement forward prop !

In [11]:
    def forward(self):
        """
        Forward propagation which will be implemented in a subclass.
        
        Compute the output value based on the 'inbound_nodes' and store the result 
        in self.value
        """
        raise NotImplemented

## Input subclass of Node

We will build subclasses of Node in order to perform calculations and hold values. For example, the **Input** subclass of Node. An input node has no inbound nodes, so there is no need to pass anything to the Node instantiator.

**NOTE** *Input node is the only node where the value may be passed as an argument to forward(). All other node implementations should get the value of the previous node from self.inbound_nodes. Example: val0 = self.inbound_nodes[0].value*

***The Input subclass does not actually calculate anything. The input subclass just holds a value, such as a data feature or a model parameter (weight/bias)***

You can set value either explicitly or with the forward() method. This value is then fed through the rest of the neural network. 

In [8]:
class Input(Node):
    def __init__(self):
        Node.__init__(self)
        
    def forward(self, value=None):
        # Overwrite the value if one is passed in
        if value is not None:
            self.value = value

## Add subclass of Node

Will perform a calculation (addition). Unlike the **Input** class, which has no inbound nodes, the **Add** class takes 2 inbound nodes, *x* and *y*, and adds the values of those nodes.

In [15]:
class Add(Node):
    def __init__(self, x, y):
        #We will access 'x' and 'y' in forward method with self.inbound_nodes[0]('x') and
        #self.inbound_nodes[1]('y')
        Node.__init__(self, [x, y])
        
    def forward(self):
        """
        We will set self.value to the sum of its inbound_nodes
        """
        self.value = self.inbound_nodes[0].value + self.inbound_nodes[1].value

## Topological sorting

For defining the network we will need to order the operations for the nodes. Given that the input to some node depends on the outputs of others, we will need to flatten the graph in such a way where all the input dependencies for each node are resolved before trying to run its calculation. This is a technique called a **topological sort**. 

For this purpose, we will implement **Kahn's Algorithm**. The method *topological_sort()* will return a sorted list of nodes in which all of the calculations can run in series. The input is a feed_dict represented by the Python dictionary data structure. 

### Implementation of the Kahn's Algorithm

In [16]:
def topological_sort(feed_dict):
    '''
    'feed_dict': A dictionary where the key is a 'Input' node and the value is the
    respective value feed to that node.
    
    Returns a list of sorted nodes.
    '''
    
    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L

Implementation for performing a forward pass through a list of sorted nodes

In [18]:
def forward_pass(output_node, sorted_nodes):
    """
    Arguments:

        'output_node': A node in the graph, should be the output node (have no outgoing edges).
        'sorted_nodes': A topologically sorted list of nodes.

    Returns the output Node's value
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.value

## Script for building and run a graph with miniflow

In [19]:
x, y = Input(), Input()

f = Add(x, y)

feed_dict = {x: 10, y: 5}

sorted_nodes = topological_sort(feed_dict)
output = forward_pass(f, sorted_nodes)

print("{} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], output))

10 + 5 = 15 (according to miniflow)
